In [1]:
from typing import Dict, List, Tuple, Union, Any, Optional

import os
import sys
print("Python version", sys.version)

# Ignore pandas warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from pathlib import Path

import pandas as pd
import numpy as np
import tensorflow as tf

import ray
from ray import train, tune

from src.datasets import daocensus

%load_ext autoreload
%autoreload 2

tf.get_logger().setLevel('WARNING')

tf.config.list_physical_devices("GPU")
sys_details = tf.sysconfig.get_build_info()
cuda = sys_details.get("cuda_version", -1)
cudnn = sys_details.get("cudnn_version", -1)
print(cuda, cudnn)

Python version 3.9.18 (main, Oct 24 2023, 09:18:18) 
[GCC 11.4.0]


2023-11-10 20:18:16.161792: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-10 20:18:16.166909: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-10 20:18:16.166940: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


11.8 8


In [2]:
# Others config
SEED: int = 57
RAY_RESULTS_PATH: Path = Path('~/ray_results').expanduser()

# Dataset splits config
N_SPLITS: int = 10
SKIP_SPLIT: int = 0

# Training config
MAX_EPOCHS: int = 200
EPOCHS_PER_ITER: int = 5

# Eval config
TOP_K: int = 5
METRICS: List[str] = ["recall", "ndcg", "precision", "map"]

# Load data

In [3]:
dfptext = pd.read_csv('./snapshot_proposals.csv')[['proposal_id', 'title', 'description', 'start', 'end']]
dfv, dfp = daocensus.get("./data/daos-census", 'Decentraland', 'snapshot')
dfv['voter'] = dfv['voter'].astype('str')
dfp = dfp.merge(dfptext, how='left', left_on='platform_proposal', right_on='proposal_id')
dfp[['start', 'end']] = dfp[['start', 'end']].astype('datetime64')
print(dfv.info())
print(dfp.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116560 entries, 0 to 116559
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   platform       116560 non-null  object        
 1   name           116560 non-null  object        
 2   id             116560 non-null  object        
 3   proposal       116560 non-null  category      
 4   deployment     116560 non-null  object        
 5   platform_vote  116560 non-null  object        
 6   voter          116560 non-null  object        
 7   date           116560 non-null  datetime64[ns]
 8   choice         116560 non-null  object        
 9   weight         116560 non-null  float64       
dtypes: category(1), datetime64[ns](1), float64(1), object(7)
memory usage: 8.3+ MB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1942 entries, 0 to 1941
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype         
---  ------ 

## Transform data

In [4]:
def to_microsoft(dfv):
    df = dfv[['voter', 'proposal', 'date']].rename(columns={
        'voter': 'userID',
        'proposal': 'itemID',
        'date': 'timestamp',
    })
    df['itemID'] = df['itemID'].astype('str')
    df['rating'] = 1
    return df

df = to_microsoft(dfv)
df.head()

,userID,itemID,timestamp,rating
0,0xe7af1c70f8f089c4c3bd71999692c6c5a15d9e2a,b86aa059-3d31-5d41-a472-70962816f779,2021-12-17 12:28:01,1
1,0xc54a6c3778016b06cbd126ccc3b5bc06c5f666fb,b86aa059-3d31-5d41-a472-70962816f779,2021-12-17 02:16:23,1
2,0xd82d005e8f8d5385db40ba23884a5c967bb1e8af,b86aa059-3d31-5d41-a472-70962816f779,2021-12-17 00:38:22,1
3,0xf4c64db66ffb301985f5ecd85c8f3f9c02f2659d,b86aa059-3d31-5d41-a472-70962816f779,2021-12-16 18:47:08,1
4,0xd5e9ef1cedad0d135d543d286a2c190b16cbb89e,b86aa059-3d31-5d41-a472-70962816f779,2021-12-16 18:32:15,1


# Split data

Each proposal remains open for a few days, our environment is different of a movies recommender system. For this reason, we will use a TimeSeriesSplit instead of a K-Fold to cross-validate the model.

![](https://scikit-learn.org/stable/_images/sphx_glr_plot_cv_indices_013.png)

El TimeSeriesSplit de scikit-learn no nos vale porque el número de elementos en cada split es el mismo, pero el tamaño del intervalo, no. Como queremos simular un comportamiento realista, haremos el split dividiendo por intervalos de igual longitud.

In [5]:
from recommenders.evaluation.python_evaluation import metrics as metrics_dict

In [6]:
from src.model_selection import timeIntervalSplit, timeIntervalSplitCurrent

max_train_prev = df['timestamp'].min().date()
folds = list(timeIntervalSplitCurrent(df, N_SPLITS, dfp, skip=SKIP_SPLIT, remove_not_in_train_col='userID'))
for i, (dftrain, dftest) in enumerate(folds):
    min_train = dftrain['timestamp'].min().date()
    max_train = dftrain['timestamp'].max().date()
    min_test  = dftest['timestamp'].min().date()
    max_test  = dftest['timestamp'].max().date()
    train_diff = (max_train-max_train_prev).days
    test_diff = (max_test-min_test).days

    train_users = len(set(dftrain['userID']))
    test_users = len(set(dftest['userID']))
    
    print(f"Split {i}, train from: {max_train_prev} to {max_train}, test from: {min_test} to {max_test}")
    print(f"  len(train): {len(dftrain)}, len(test): {len(dftest)}")
    print(f"  users(train): {train_users}, users(test): {test_users}")

    print()
    dftest['prediction'] = 1
    for m in METRICS:
        f = metrics_dict[f'{m}_at_k']
        print(f"  highest possible {m}@{TOP_K}:\t{f(dftest, dftest, k=TOP_K, relevancy_method='top_k'):.4f}")

    print("-"*30)

    max_train_prev = max_train

Split 0, train from: 2021-05-24 to 2021-08-04, test from: 2021-08-04 to 2021-08-10
  len(train): 3453, len(test): 48
  users(train): 581, users(test): 27

  highest possible recall@5:	0.9894
  highest possible ndcg@5:	1.0000
  highest possible precision@5:	0.3407
  highest possible map@5:	0.9894
------------------------------
Split 1, train from: 2021-08-04 to 2021-10-14, test from: 2021-10-15 to 2021-10-19
  len(train): 6397, len(test): 132
  users(train): 930, users(test): 121

  highest possible recall@5:	0.9976
  highest possible ndcg@5:	1.0000
  highest possible precision@5:	0.2149
  highest possible map@5:	0.9976
------------------------------
Split 2, train from: 2021-10-14 to 2021-12-25, test from: 2021-12-25 to 2021-12-31
  len(train): 15187, len(test): 99
  users(train): 2388, users(test): 47

  highest possible recall@5:	0.9721
  highest possible ndcg@5:	1.0000
  highest possible precision@5:	0.3660
  highest possible map@5:	0.9721
------------------------------
Split 3, tra

# Defining training

In [7]:
from recommenders.models.deeprec.models.graphrec.lightgcn import LightGCN

class LightGCNCustom(LightGCN):
    # Copied from LightGCN.fit but RETURNING the data and deleting unnecessary things
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.epochs_done = 0
    
    def fit_epoch(self):
        """Fit the model on self.data.train. If eval_epoch is not -1, evaluate the model on `self.data.test`
        every `eval_epoch` epoch to observe the training status.

        """
        loss, mf_loss, emb_loss = 0.0, 0.0, 0.0
        n_batch = self.data.train.shape[0] // self.batch_size + 1
        for idx in range(n_batch):
            users, pos_items, neg_items = self.data.train_loader(self.batch_size)
            _, batch_loss, batch_mf_loss, batch_emb_loss = self.sess.run(
                [self.opt, self.loss, self.mf_loss, self.emb_loss],
                feed_dict={
                    self.users: users,
                    self.pos_items: pos_items,
                    self.neg_items: neg_items,
                },
            )
            loss += batch_loss / n_batch
            mf_loss += batch_mf_loss / n_batch
            emb_loss += batch_emb_loss / n_batch

        if np.isnan(loss):
            print("ERROR: loss is nan.")
            sys.exit()

        self.epochs_done += 1

        return loss, mf_loss, emb_loss

In [8]:
from recommenders.models.deeprec.deeprec_utils import prepare_hparams
from recommenders.models.deeprec.DataModel.ImplicitCF import ImplicitCF

class TrainLightGCN(tune.Trainable):
    def setup(
        self,
        config: Dict[str, Any],
        folds: List[Tuple[pd.DataFrame, pd.DataFrame]],
    ):
        self.config = config
        config['batch_size'] = 2**config['batch_size']
        self.fold = config['__trial_index__']

        self.hparams = prepare_hparams(
            model_type='lightgcn',
            n_layers=config['conv_layers'],
            batch_size=config['batch_size'],
            embed_size=config['embedding_dim'],
            epochs=EPOCHS_PER_ITER,
            learning_rate=config['learning_rate'],
            decay=config['l2'],
            metrics=METRICS,
            eval_epoch=-1,
            top_k=TOP_K,
            save_model=False,
            MODEL_DIR='./data/model/lightgcn/',
        )

        self.dataloader = ImplicitCF(train=folds[self.fold][0], test=folds[self.fold][1], seed=SEED)
        self.model = LightGCNCustom(self.hparams, self.dataloader, seed=SEED)

    @property
    def iteration(self):
        return self.model.epochs_done

    @property
    def training_iteration(self):
        return self.model.epochs_done

    def step(self):
        """
        As a rule of thumb, the execution time of step should be large enough to avoid overheads 
        (i.e. more than a few seconds), but short enough to report progress periodically 
        (i.e. at most a few minutes).
        """
        assert EPOCHS_PER_ITER > 0
        
        for _ in range(EPOCHS_PER_ITER):
            ret = self.model.fit_epoch()

        return {
            'iteration': self.iteration,
            'loss': ret[0],
            'mf_loss': ret[1],
            'emb_loss': ret[2],
            **{k:v for k,v in zip(self.model.metrics, self.model.run_eval())},
        }

    def save_checkpoint(self, checkpoint_dir):
        checkpoint_path = os.path.join(checkpoint_dir, "model")
        self.model.saver.save(
            sess=self.model.sess,
            save_path=checkpoint_path,
        )
        return checkpoint_dir

    def load_checkpoint(self, checkpoint_path):
        self.model.load(checkpoint_path)

### Small test

In [9]:
hparams = prepare_hparams(
    model_type='lightgcn',
    n_layers=3,
    batch_size=512,
    embed_size=64,
    epochs=10,
    learning_rate=0.001,
    decay=0.001,
    metrics=["recall", "ndcg", "precision", "map"],
    eval_epoch=2,
    top_k=TOP_K,
    save_model=False,
    MODEL_DIR='./data/model/lightgcn/',
)
dataloader = ImplicitCF(train=folds[-1][0], test=folds[-1][1], seed=SEED)
print("items:", dataloader.n_items, "user:", dataloader.n_users)
model = LightGCNCustom(
    hparams,
    dataloader,
    seed=SEED,
)

items: 1836 user: 6857
Already create adjacency matrix.
Already normalize adjacency matrix.
Using xavier initialization.


In [10]:
model.fit()

Epoch 1 (train)3.9s: train loss = 0.60378 = (mf)0.60305 + (embed)0.00073
Epoch 2 (train)1.9s + (eval)3.2s: train loss = 0.35138 = (mf)0.34746 + (embed)0.00392, recall = 0.00400, ndcg = 0.00394, precision = 0.00320, map = 0.00200
Epoch 3 (train)3.3s: train loss = 0.25582 = (mf)0.24906 + (embed)0.00676
Epoch 4 (train)2.9s + (eval)0.2s: train loss = 0.20802 = (mf)0.19926 + (embed)0.00876, recall = 0.00400, ndcg = 0.00394, precision = 0.00320, map = 0.00200
Epoch 5 (train)2.3s: train loss = 0.18292 = (mf)0.17263 + (embed)0.01029
Epoch 6 (train)1.0s + (eval)0.2s: train loss = 0.16253 = (mf)0.15102 + (embed)0.01151, recall = 0.00400, ndcg = 0.00394, precision = 0.00320, map = 0.00200
Epoch 7 (train)3.2s: train loss = 0.14699 = (mf)0.13448 + (embed)0.01250
Epoch 8 (train)3.2s + (eval)0.2s: train loss = 0.13641 = (mf)0.12309 + (embed)0.01332, recall = 0.00453, ndcg = 0.00730, precision = 0.00480, map = 0.00411
Epoch 9 (train)3.0s: train loss = 0.12787 = (mf)0.11383 + (embed)0.01404
Epoch 10 (t

### Big experiment

In [11]:
os.uname().nodename

'lamarck'

In [12]:
RAY_RESULTS_PATH

PosixPath('/home/daviddavo/ray_results')

In [13]:
import ray
from ray.tune.search.hyperopt import HyperOptSearch

last_experiment = max(RAY_RESULTS_PATH.glob('TrainLightGCN_*'), key=lambda x: x.stat().st_ctime)
# last_experiment = None

### SET TRAINING RESOURCES
if os.uname().nodename == 'lamarck':
    # assert torch.cuda.is_available()
    
    NUM_SAMPLES = 100
    # Every run takes approx half a gig of vram (no optimizations)
    # The RTX 4090 has 24GB so we can run the model about 48 times
    resources_per_trial={
        'cpu': 1,
        # GPU has 25GiB, and each run might take up to 2GiB (torch version was lighter)
        # so each run might take up to 1/12th of the GPU
        # I use 1/8th so I don't take all the resources in the machine
        'gpu': 1/8,
    }
else:
    NUM_SAMPLES = 1
    resources_per_trial={
        'cpu': 1,
        # It takes about 1.5 GiB with full training data, but I put a bit more because
        # this notebook also takes a bit of memory
        'memory': 2e9,
    }

### RESTORE EXPERIMENT OR CREATE A NEW ONE
if last_experiment and tune.Tuner.can_restore(last_experiment):
    print(f"Restoring last experiment: {last_experiment}")
    tuner = tune.Tuner.restore(
        str(last_experiment),
        trainable=tune.with_resources(
            tune.with_parameters(TrainLightGCN, folds=folds),
            resources_per_trial,
        ),
        restart_errored=True
    )
else:
    print("No experiment found, creating new tuner")
    search_alg = HyperOptSearch()
    search_alg = tune.search.Repeater(search_alg, N_SPLITS-SKIP_SPLIT)
    
    tuner = tune.Tuner(
        tune.with_resources(
            tune.with_parameters(TrainLightGCN, folds=folds),
            resources_per_trial,
        ),
        run_config=train.RunConfig(
            stop={'training_iteration': MAX_EPOCHS/EPOCHS_PER_ITER},
            storage_path=RAY_RESULTS_PATH,
        ),
        param_space=dict(
            # batch size between 2**4 (32) and 2**10 (1024)
            batch_size=tune.randint(4,10),
            embedding_dim=tune.qlograndint(10, 500, 5),
            conv_layers=tune.randint(2,6),
            learning_rate=tune.qloguniform(1e-4, 1, 1e-4),
            l2=tune.loguniform(1e-7, 1e-1),
        ),
        tune_config=tune.TuneConfig(
            search_alg=search_alg,
            num_samples=(N_SPLITS-SKIP_SPLIT)*NUM_SAMPLES,
            metric='loss',
            mode='min',
        )
    )

No experiment found, creating new tuner


In [ ]:
tuner.fit()

(pid=997590) 2023-11-10 20:20:39.496410: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
(pid=997590) 2023-11-10 20:20:39.496444: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(pid=997590) 2023-11-10 20:20:39.496458: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
(TrainLightGCN pid=997590) /home/daviddavo/upm-tfm-notebooks/.direnv/python-3.9/lib/python3.9/site-packages/recommenders/models/deeprec/DataModel/ImplicitCF.py:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
(TrainLightGCN pid=997590)

(TrainLightGCN pid=997590) Already create adjacency matrix.
(TrainLightGCN pid=997590) Already normalize adjacency matrix.
(TrainLightGCN pid=997590) Using xavier initialization.


(TrainLightGCN pid=997590) Trainable.setup took 34.618 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
(pid=998387) 2023-11-10 20:21:59.025681: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
(pid=998387) 2023-11-10 20:21:59.025718: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(pid=998387) 2023-11-10 20:21:59.025730: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
(TrainLightGCN pid=998387) /home/daviddavo/upm-tfm-notebooks/.direnv/python-3.9/lib/python3.9/site-packages/recommenders/models/deeprec/DataModel/I

(TrainLightGCN pid=998387) Already create adjacency matrix.
(TrainLightGCN pid=998387) Already normalize adjacency matrix.
(TrainLightGCN pid=998387) Using xavier initialization.


(TrainLightGCN pid=998387) Trainable.setup took 23.086 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
(pid=999049) 2023-11-10 20:23:08.530072: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
(pid=999049) 2023-11-10 20:23:08.530107: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(pid=999049) 2023-11-10 20:23:08.530119: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
(TrainLightGCN pid=999049) /home/daviddavo/upm-tfm-notebooks/.direnv/python-3.9/lib/python3.9/site-packages/recommenders/models/deeprec/DataModel/I

...

(TrainLightGCN pid=1063793) Already normalize adjacency matrix.
(TrainLightGCN pid=1063793) Using xavier initialization.
(TrainLightGCN pid=1063793) Already create adjacency matrix.


(TrainLightGCN pid=1060997) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/home/daviddavo/ray_results/TrainLightGCN_2023-11-10_20-19-58/TrainLightGCN_f97751a8_247_trial_index=6,batch_size=6,conv_layers=2,embedding_dim=155,l2=0.0000,learning_rate=0.0009_2023-11-11_08-28-54/checkpoint_000000)
(pid=1063955) 2023-11-11 08:53:13.623532: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
(pid=1063955) 2023-11-11 08:53:13.623569: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(pid=1063955) 2023-11-11 08:53:13.623582: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
(Tra

(TrainLightGCN pid=1063955) Already create adjacency matrix.
(TrainLightGCN pid=1063955) Already normalize adjacency matrix.
(TrainLightGCN pid=1063955) Using xavier initialization.


(TrainLightGCN pid=1063793) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/home/daviddavo/ray_results/TrainLightGCN_2023-11-10_20-19-58/TrainLightGCN_8375f4ca_252_trial_index=1,batch_size=6,conv_layers=2,embedding_dim=490,l2=0.0000,learning_rate=0.0014_2023-11-11_08-51-27/checkpoint_000000)
(pid=1064328) 2023-11-11 08:55:54.652917: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
(pid=1064328) 2023-11-11 08:55:54.652952: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(pid=1064328) 2023-11-11 08:55:54.652963: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
(Tra

(TrainLightGCN pid=1064328) Already normalize adjacency matrix.
(TrainLightGCN pid=1064328) Using xavier initialization.
(TrainLightGCN pid=1064328) Already create adjacency matrix.
